In [ ]:
library(Matrix)
library(Seurat)
library(dplyr)

In the demonstration in Fig. 5, we performed INTERSTELLAR translation of reads from each subregion independently. Thus, we have SpaceRanger results for each subregion and need to combine the transcriptome information into a single matrix.

In [ ]:
cnt <- 0
dircount <- 0
for(tissue_sample in c("hippo","cereb1","cereb2","kidney","liver")){
    dirs <- list.files(paste0("/workdir/",tissue_sample,"/SpaceRanger/"))
    for(dir in dirs){
        if(cnt==0){
            outdir <- paste0("/workdir/",tissue_sample,"/SpaceRanger/",
                             dir,
                             "/outs/filtered_feature_bc_matrix")
            dge <- Read10X(outdir)
            dge <- dge[,colSums(dge)>0]
            colnames(dge) <- paste0(dir,"_",colnames(dge))
            cnt <- 1
            print(c(dir,ncol(dge)))
        }else{
            outdir <- paste0("/workdir/",tissue_sample,"/SpaceRanger/",
                             dir,
                             "/outs/filtered_feature_bc_matrix")
            dge.tmp0 <- Read10X(outdir)
            dge.tmp <- dge.tmp0[,colSums(dge.tmp0)>0]
            cname <- colnames(dge.tmp0)[colSums(dge.tmp0)>0]
            print(c(dir,ncol(dge.tmp)))
            if(length(cname)==1){
                print(c(dir,"true"))
                dge <- cbind(dge,dge.tmp)
                colnames(dge)[ncol(dge)] <- paste0(dir,"_",cname)
            }else{
                colnames(dge.tmp) <- paste0(dir,"_",colnames(dge.tmp))
                dge <- cbind(dge,dge.tmp)
            }
        }

    }
    write.table(as.matrix(dge),paste0("slide_",tissue_sample,"_ed0_visium_Feb2021.csv"),quote=F,sep=",")
}